In [1]:
#Import necessary packages
import pandas as pd
import numpy as np
import re
import gc
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
train = pd.read_csv('train.csv')
train

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate
...,...,...,...,...,...
36760,9f63b687e76a,FFA381E58FC6,For many people they don't like only asking on...,Claim,Adequate
36761,9d5bd7d86212,FFA381E58FC6,also people have different views and opinions ...,Claim,Adequate
36762,f1b78becd573,FFA381E58FC6,Advice is something that can impact a persons ...,Position,Adequate
36763,cc184624ca8e,FFA381E58FC6,someone can use everything that many people sa...,Evidence,Ineffective


In [3]:
#Clean the text of punctuation, line breaks, capitalization, etc
def clean_text(contents):
    contents = re.sub("'", "", contents)
    cleaned_contents = re.sub('[^A-Za-z]+', ' ', contents)
    return re.sub('/\r?\n|\r/g', ' ', cleaned_contents).lower()

train.discourse_text = train.discourse_text.transform(clean_text)

In [4]:
#Find the count of each word and create a document-term matrix
cv = CountVectorizer(stop_words = 'english')
text_cv = cv.fit_transform(train.discourse_text)
dtm = pd.DataFrame(text_cv.toarray(), columns = cv.get_feature_names())

In [6]:
#Drop words that are  present in less than 1% of the text samples
word_counts = dtm.sum().where(dtm.sum() > (0.01*len(dtm.index)))
word_counts.dropna(inplace = True)
dtm = dtm.loc[:, word_counts.index.array]

In [10]:
#Index samples by discourse ID to ensure compatability with original training dataset
dtm['discourse_id'] = train['discourse_id']
dtm.to_csv('dtm.csv', header = True, index = False)

In [8]:
gc.collect()

75